In [1]:
from ns3gym import ns3env
from comet_ml import Experiment
import tqdm
import subprocess
from collections import deque
import numpy as np

from agents.ddpg.agent import Agent
from agents.teacher import Teacher

C:\ProgramData\Anaconda3\lib\site-packages\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Basic constants and setting up environment

In [2]:
simTime = 10 # seconds
stepTime = 0.1  # seconds

EPISODE_COUNT = 15
steps_per_ep = int(simTime/stepTime)

SCRIPT_RUNNING = False

script_exec_command = f'../../waf --run "linear-mesh --simTime={simTime} --envStepTime={stepTime}"'
print("Steps per episode:", steps_per_ep)

Steps per episode: 100


In [3]:
port = 5555
seed = 0

simArgs = {"--simTime": simTime,
           "--testArg": 123,
           "--nodeNum": 5,
           "--distance": 500}
debug = False

subprocess.Popen(['bash', '-c', script_exec_command])
SCRIPT_RUNNING = True

env = ns3env.Ns3Env(port=port, stepTime=stepTime, startSim=0, simSeed=seed, simArgs=simArgs, debug=debug)

Waiting for simulation script to connect on port: tcp://localhost:5555
Please start proper ns-3 simulation script using ./waf --run "..."


In [4]:
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

assert ob_space is not None

Observation space shape: Box(1,)
Action space shape: Box(1,)


### Creating and training agent

In [5]:
agent = Agent(1, 1)
teacher = Teacher(agent, env, 1)
teacher.train(EPISODE_COUNT, simTime, stepTime, "Inp: Mb sent", "Rew: improvement", "Small net")

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/wwydmanski/rl-in-wifi/be32ff66572b4c0a938ff9b180d7985a

 67%|█████████████████████████████████████████▌                    | 67/100 [00:46<00:23,  1.43it/s, mb_sent=217.63 Mb]

------- STARTED TRAINING -------


100%|█████████████████████████████████████████████████████████████| 100/100 [01:14<00:00,  1.35it/s, mb_sent=347.52 Mb]


Sent 347.52 Mb/s.	Mean per round: 3.48 Mb/s	Episode 1/15 finished

Waiting for simulation script to connect on port: tcp://localhost:5555
Please start proper ns-3 simulation script using ./waf --run "..."


100%|█████████████████████████████████████████████████████████████| 100/100 [01:11<00:00,  1.41it/s, mb_sent=351.49 Mb]


...

Sent 319.26 Mb/s.	Mean per round: 3.19 Mb/s	Episode 14/15 finished

Waiting for simulation script to connect on port: tcp://localhost:5555
Please start proper ns-3 simulation script using ./waf --run "..."


100%|█████████████████████████████████████████████████████████████| 100/100 [00:59<00:00,  1.69it/s, mb_sent=312.18 Mb]


Sent 312.18 Mb/s.	Mean per round: 3.12 Mb/s	Episode 15/15 finished



COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Metrics:
COMET INFO:                     Chosen CW: 178.57847567448118
COMET INFO:             Cumulative reward: 140.81337022781372
COMET INFO:                Megabytes sent: 312.18336
COMET INFO:                 Per-ep reward: 140.81337022781372
COMET INFO:                  Round reward: -30.556608200073242
COMET INFO:                         Speed: 31.218336
COMET INFO:                    actor_loss: -0.68321955
COMET INFO:                   critic_loss: 1541.9777
COMET INFO:                          loss: -0.6832195520401001
COMET INFO:         sys.gpu.0.free_memory: 8298512384
COMET INFO:     sys.gpu.0.gpu_utilization: 3
COMET INFO:        sys.gpu.0.total_memory: 11811160064
COMET INFO:         sys.gpu.0.used_memory: 3512647680
COMET INFO: ----------------------------
COMET INFO: Uploading stats to Comet before program termination (may take several seconds)


Training finished.
